In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds

In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [4]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

In [5]:
def normalize_img(image, label):
    return tf.cast(image, tf.float32)/255.0, label

In [6]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 32

ds_train = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE).cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(AUTOTUNE)

ds_test = ds_test.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE)
ds_test = ds_test.prefetch(AUTOTUNE)

In [23]:
model = keras.Sequential(
    [
        layers.Input(shape=(28,28,1)),
        layers.Conv2D(32, 3, activation='relu'),
        layers.Flatten(),
        layers.Dense(10)
    ]
)

In [24]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 flatten_2 (Flatten)         (None, 21632)             0         
                                                                 
 dense_2 (Dense)             (None, 10)                216330    
                                                                 
Total params: 216,650
Trainable params: 216,650
Non-trainable params: 0
_________________________________________________________________


In [25]:
save_callback = keras.callbacks.ModelCheckpoint(
    'models/',
    save_weights_only=True,
    save_best_only=False,
    monitor='accuracy'
)

In [26]:
def scheduler(epoch, lr):
  if epoch < 10:
    return lr
  else:
    return lr * tf.math.exp(-0.1)

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [27]:
class CustomCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs.get('accuracy') > 0.99:
            print('\nAccuracy over 99%, quitting training')
            self.model.stop_training = True

In [28]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(learning_rate=0.001),
    metrics = ['accuracy']
)

In [29]:
model.fit(
    ds_train,
    epochs=10,
    validation_data=ds_test,
    callbacks=[save_callback, lr_scheduler, CustomCallback()]
);

Epoch 1/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.1785 - accuracy: 0.9471 - val_loss: 0.0775 - val_accuracy: 0.9771 - lr: 0.0010
Epoch 2/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0692 - accuracy: 0.9793 - val_loss: 0.0629 - val_accuracy: 0.9790 - lr: 0.0010
Epoch 3/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0506 - accuracy: 0.9846 - val_loss: 0.0678 - val_accuracy: 0.9777 - lr: 0.0010
Epoch 4/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0372 - accuracy: 0.9885 - val_loss: 0.0604 - val_accuracy: 0.9805 - lr: 0.0010
Epoch 5/10
1872/1875 [============================>.] - ETA: 0s - loss: 0.0284 - accuracy: 0.9908
Accuracy over 99%, quitting training
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0284 - accuracy: 0.9908 - val_loss: 0.0707 - val_accuracy: 0.9797 - lr: 0.0010
